In [1]:
# Init
import glob
import os
import pickle
import sys

module_path = os.path.abspath(os.path.join("../src/simulicronalpha/"))
if module_path not in sys.path:
    sys.path.append(module_path)

import multiprocessing
import warnings

# Imports
import numpy as np
import pandas as pd

np.set_printoptions(suppress=True)
# For loading data
import concurrent.futures
import gc
import json
import random

# Plotting imports
import altair as alt
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib.pyplot import figure
from numpy import concatenate as c
from numpy import cumsum

In [2]:
# Acc. functions
def returnLast(lst):
    return lst[-1]


def returnGen(lst):
    return lst[1500]


def returnList(HGTGen, lst):
    return lst[HGTGen]


def colorRanges(i):
    # Construct the colormap
    current_palette = sns.color_palette("muted", n_colors=5)
    cmap = ListedColormap(sns.color_palette(current_palette).as_hex())
    # range and stuff
    if i == 0:
        return "0"
    elif i > 0 and i <= 20:
        return "1-20"
    elif i > 20 and i <= 50:
        return "20-50"
    elif i > 50 and i <= 100:
        return "50-100"
    elif i > 100 and i <= 200:
        return "100-200"
    elif i > 200 and i <= 300:
        return "200-300"
    elif i > 300:
        return ">300"


# Since pickle format is not working as intended.
# Insert angry thoughts abouts pickle format here
# A function to open pickle files and convert into json
def pickleToJson(resultDir):
    for file in glob.glob(os.path.join(resultDir, "*.pickle")):
        with open(file, "rb") as f:
            # print ("Converting file : " + file)
            gc.disable()
            data = pickle.load(f)
            del data["TEset"]
            del data["NTE"]
            del data["NTI"]
            del data["AvgFit"]
            del data["Freq"]
            gc.enable()
            with open(file + ".json", "w") as fp:
                json.dump(data, fp)
    return 0


def bin_by(x, y, nbins=30, bins=None):
    if bins is None:
        bins = np.linspace(x.min(), x.max(), nbins)

    bin_space = (bins[-1] - bins[0]) / (len(bins) - 1) / 2

    indicies = np.digitize(x, bins + bin_space)

    output = []
    for i in range(0, len(bins)):
        output.append(y[indicies == i])
    #
    # prepare a dataframe with cols: median; mean; 1up, 1dn, 2up, 2dn, 3up, 3dn
    df_names = ["mean", "median", "5th", "95th", "10th", "90th", "25th", "75th"]
    df = pd.DataFrame(columns=df_names)
    to_delete = []
    # for each bin, determine the std ranges
    for y_set in output:
        if y_set.size > 0:
            av = y_set.mean()
            intervals = np.percentile(y_set, q=[50, 5, 95, 10, 90, 25, 75])
            res = [av] + list(intervals)
            df = df.append(pd.DataFrame([res], columns=df_names))
        else:
            # just in case there are no elements in the bin
            to_delete.append(len(df) + 1 + len(to_delete))

    # add x values
    bins = np.delete(bins, to_delete)
    df["x"] = bins

    return df


def dfWrangler(dataFrame):
    # Perform some data wrangling
    dataFrame[["TranspositionRate_1", "TranspositionRate_2"]] = pd.DataFrame(
        dataFrame.TRate.tolist(), index=dataFrame.index
    )
    dataFrame[["TE_1_CN", "TE_2_CN"]] = dataFrame["TEfamilyCN"].apply(pd.Series)
    dataFrame[["TE_1_RG", "TE_2_RG"]] = dataFrame["TEfamilyRg"].apply(pd.Series)
    dataFrame[["TE_1_pi", "TE_2_pi"]] = dataFrame["TEpi"].apply(pd.Series)

    # Add column for the last acn (average copy number) for each transposon family
    dataFrame["TE_1_ACN"] = dataFrame["TE_1_CN"].apply(lambda x: returnLast(x))
    dataFrame["TE_2_ACN"] = dataFrame["TE_2_CN"].apply(lambda x: returnLast(x))

    # Add column for copies not present in clusters
    dataFrame["TE_1_NCN"] = dataFrame["TE_1_ACN"] - dataFrame["TE_1_pi"]
    dataFrame["TE_2_NCN"] = dataFrame["TE_2_ACN"] - dataFrame["TE_2_pi"]
    dataFrame["TE_1_NCN_l"] = np.log(dataFrame["TE_1_ACN"] - dataFrame["TE_1_pi"])
    dataFrame["TE_2_NCN_l"] = np.log(dataFrame["TE_2_ACN"] - dataFrame["TE_2_pi"])

    # Add column for nth generation copy number
    dataFrame["TE_1_ACN_N"] = dataFrame["TE_1_CN"].apply(lambda x: returnGen(x))
    dataFrame["TE_2_ACN_N"] = dataFrame["TE_2_CN"].apply(lambda x: returnGen(x))

    # Get ranges for adding colors later
    dataFrame["TE_1_NCN_R"] = dataFrame["TE_1_NCN"].apply(lambda x: colorRanges(x))
    dataFrame["TE_2_NCN_R"] = dataFrame["TE_2_NCN"].apply(lambda x: colorRanges(x))

    # Some wrangling for plotly go
    # x = dataFrame["ETA"]
    # y = dataFrame["selPen"]
    # TE1ACN = dataFrame["TE_1_ACN"]
    # TE2ACN = dataFrame["TE_2_ACN"]

    # Convert count to log scale
    # resultDataframeC["TE_1_ACN_l"] = np.log(resultDataframeC["TE_1_ACN"])
    # resultDataframeC["TE_2_ACN_l"] = np.log(resultDataframeC["TE_2_ACN"])
    return dataFrame

In [3]:
current_palette = sns.color_palette("muted", n_colors=5)
cmap = ListedColormap(sns.color_palette(current_palette).as_hex())

In [4]:
# Path to results
resultDir = "/home/siddharth/git/Simulicron/Project_1C/coreSim/"

# Convert them to json
# returnValue = pickleToJson(resultDir)

In [5]:
# Read the json files into pandas dataframe
SimulationRecords = []
for filename in glob.glob(os.path.join(resultDir, "*_6.pickle.json")):
    with open(filename, "r") as file:
        SimulationRecords.append(json.load(file))

SimulationResultsDataFrame = pd.DataFrame(SimulationRecords)

# Create a copy for inplace handling
resultDataframe_1 = SimulationResultsDataFrame.copy(deep=True)
resultDataframe_1 = dfWrangler(resultDataframe_1)

/home/siddharth/mambaforge/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
# Read the json files into pandas dataframe
SimulationRecords = []
for filename in glob.glob(os.path.join(resultDir, "*_3.pickle.json")):
    with open(filename, "r") as file:
        SimulationRecords.append(json.load(file))

SimulationResultsDataFrame = pd.DataFrame(SimulationRecords)

# Create a copy for inplace handling
resultDataframe_2 = SimulationResultsDataFrame.copy(deep=True)
resultDataframe_2 = dfWrangler(resultDataframe_2)

In [7]:
# Extract data
# For #1 TE
TE_1_ACN_1 = np.asarray(resultDataframe_1["TE_1_ACN"].tolist())
# np.log(TE_1_ACN_1, out=TE_1_ACN_1, where=TE_1_ACN_1 > 0)
ETA_1 = np.asarray(resultDataframe_1["ETA"].tolist())


# For #2 TE
TE_1_ACN_2 = np.asarray(resultDataframe_2["TE_1_ACN"].tolist())
TE_2_ACN_2 = np.asarray(resultDataframe_2["TE_2_ACN"].tolist())

# np.log(TE_1_ACN_2, out=TE_1_ACN_2, where=TE_1_ACN_2 > 0)
# np.log(TE_2_ACN_2, out=TE_2_ACN_2, where=TE_2_ACN_2 > 0)

ETA_2 = np.asarray(resultDataframe_2["ETA"].tolist())

# Process them
# Alpha
df_1_Alpha = bin_by(ETA_1, TE_1_ACN_1, nbins=25, bins=None)
# Alpha+Beta
df_2_Alpha = bin_by(ETA_2, TE_1_ACN_2, nbins=25, bins=None)
df_2_Beta = bin_by(ETA_2, TE_2_ACN_2, nbins=25, bins=None)

In [8]:
figure(figsize=(5, 5), dpi=150)
plt.clf()
with plt.style.context("ggplot"):
    cols = ["#EE7550", "#F19463", "#F6B176"]
    '''
    # plot the 3rd stdv
    plt.fill_between(
        df_1_Alpha.x, df_1_Alpha["5th"], df_1_Alpha["95th"], alpha=0.1, color=cols[2]
    )
    plt.fill_between(
        df_1_Alpha.x, df_1_Alpha["10th"], df_1_Alpha["90th"], alpha=0.1, color=cols[1]
    )
    plt.fill_between(
        df_1_Alpha.x, df_1_Alpha["25th"], df_1_Alpha["75th"], alpha=0.1, color=cols[0]
    )'''
    # plt the line
    plt.plot(df_1_Alpha.x, df_1_Alpha["median"], color="blue", alpha=0.7, linewidth=1, label="TE α")
    # plot the points
    plt.scatter(
        ETA_1,
        TE_1_ACN_1,
        facecolors="blue",
        edgecolors="0",
        s=5,
        lw=0.7,
        alpha=0.05,
    )

    cols = ["#B31217", "#991014", "#800D11"]
    '''
    # plot the 3rd stdv
    plt.fill_between(
        df_2_Alpha.x, df_2_Alpha["5th"], df_2_Alpha["95th"], alpha=0.1, color=cols[2]
    )
    plt.fill_between(
        df_2_Alpha.x, df_2_Alpha["10th"], df_2_Alpha["90th"], alpha=0.1, color=cols[1]
    )
    plt.fill_between(
        df_2_Alpha.x, df_2_Alpha["25th"], df_2_Alpha["75th"], alpha=0.1, color=cols[0]
    )'''
    # plt the line
    plt.plot(df_2_Alpha.x, df_2_Alpha["median"], color="orange", alpha=0.7, linewidth=1, label="TE α (with TE β)")
    # plot the points
    plt.scatter(
        ETA_2,
        TE_1_ACN_2,
        facecolors="orange",
        edgecolors="0",
        s=5,
        lw=0.7,
        alpha=0.05,
    )

    cols = ["#fa9470", "#daae9d", "#ffa591"]
    '''
    # plot the 3rd stdv
    plt.fill_between(
        df_2_Beta.x, df_2_Beta["5th"], df_2_Beta["95th"], alpha=0.1, color=cols[2]
    )
    plt.fill_between(
        df_2_Beta.x, df_2_Beta["10th"], df_2_Beta["90th"], alpha=0.1, color=cols[1]
    )
    plt.fill_between(
        df_2_Beta.x, df_2_Beta["25th"], df_2_Beta["75th"], alpha=0.1, color=cols[0]
    )'''
    # plt the line
    plt.plot(df_2_Beta.x, df_2_Beta["median"], color="green", alpha=0.7, linewidth=1, label="TE β (with TE α)")
    # plot the points
    plt.scatter(
        ETA_2,
        TE_2_ACN_2,
        facecolors="green",
        edgecolors="0",
        s=5,
        lw=0.7,
        alpha=0.05,
    )
    plt.yscale('log')
    plt.xlabel('Cross-regulation coefficient', fontsize=18)
    plt.ylabel('TE average copy number', fontsize=16)
    plt.legend(loc="lower left")
    plt.savefig("CrossRegulationBase.svg", bbox_inches = "tight")
    plt.savefig("CrossRegulationBase.pdf", bbox_inches = "tight")
    plt.show()
    

In [9]:
# Make area plots
alpha_1_cols = ["#00414d", "#384a51", "#00536d", "#456972", "#337f9e"]
chart_1_alpha_0 = (
    alt.Chart(df_1_Alpha)
    .mark_area(color=alpha_1_cols[4], opacity=0.01)
    .encode(
        x="x:Q",
        y="5th:Q",
        y2="95th:Q",
    )
)
chart_1_alpha_1 = (
    alt.Chart(df_1_Alpha)
    .mark_area(color=alpha_1_cols[3], opacity=0.01)
    .encode(
        x="x:Q",
        y="10th:Q",
        y2="90th:Q",
    )
)
chart_1_alpha_2 = (
    alt.Chart(df_1_Alpha)
    .mark_area(color=alpha_1_cols[2], opacity=0.01)
    .encode(
        x="x:Q",
        y="25th:Q",
        y2="75th:Q",
    )
)
chart_1_alpha_median = (
    alt.Chart(df_1_Alpha)
    .mark_line(color="blue", opacity=0.5)
    .encode(
        y="median",
        x="x:Q",
    )
)
chart_1_alpha_scatter_Df = pd.DataFrame(
    {
        "Average copy number (Log)": TE_1_ACN_1,
        "Epistasis Coefficient": epistasisCoefficient_1,
    }
)
chart_1_alpha_scatter = (
    alt.Chart(chart_1_alpha_scatter_Df)
    .mark_circle(color=alpha_1_cols[0], opacity=0.01)
    .encode(
        x="Epistasis Coefficient",
        y="Average copy number (Log)",
    )
)
chart_1 = (
    chart_1_alpha_0
    + chart_1_alpha_1
    + chart_1_alpha_2
    + chart_1_alpha_scatter
    + chart_1_alpha_median
)

# -------------------------#
alpha_2_cols = ["#a3432f", "#b28081", "#fa9470", "#daae9d", "#ffa591"]
chart_2_alpha_0 = (
    alt.Chart(df_2_Alpha)
    .mark_area(color=alpha_2_cols[4], opacity=0.01)
    .encode(
        x="x:Q",
        y="5th:Q",
        y2="95th:Q",
    )
)
chart_2_alpha_1 = (
    alt.Chart(df_2_Alpha)
    .mark_area(color=alpha_2_cols[3], opacity=0.01)
    .encode(
        x="x:Q",
        y="10th:Q",
        y2="90th:Q",
    )
)
chart_2_alpha_2 = (
    alt.Chart(df_2_Alpha)
    .mark_area(color=alpha_2_cols[2], opacity=0.01)
    .encode(
        x="x:Q",
        y="25th:Q",
        y2="75th:Q",
    )
)
chart_2_alpha_median = (
    alt.Chart(df_2_Alpha)
    .mark_line(color="green", opacity=0.5)
    .encode(
        y="median",
        x="x:Q",
    )
)
chart_2_alpha_scatter_Df = pd.DataFrame(
    {
        "Average copy number (Log)": TE_1_ACN_2,
        "Epistasis Coefficient": epistasisCoefficient_2,
    }
)
chart_2_alpha_scatter = (
    alt.Chart(chart_2_alpha_scatter_Df)
    .mark_point(color=alpha_2_cols[0], opacity=0.01)
    .encode(
        x="Epistasis Coefficient",
        y="Average copy number (Log)",
    )
)
chart_2 = (
    chart_2_alpha_0
    + chart_2_alpha_1
    + chart_2_alpha_2
    + chart_2_alpha_scatter
    + chart_2_alpha_median
)
# -------------------------#
beta_2_cols = ["#FF1A21", "#E6171E", "#CC151A", "#B31217", "#991014", "#800D11"]
chart_3_beta_0 = (
    alt.Chart(df_2_Beta)
    .mark_area(color=alpha_2_cols[4], opacity=0.05)
    .encode(
        x="x:Q",
        y="5th:Q",
        y2="95th:Q",
    )
)
chart_3_beta_1 = (
    alt.Chart(df_2_Beta)
    .mark_area(color=alpha_2_cols[3], opacity=0.01)
    .encode(
        x="x:Q",
        y="10th:Q",
        y2="90th:Q",
    )
)
chart_3_beta_2 = (
    alt.Chart(df_2_Beta)
    .mark_area(color=alpha_2_cols[2], opacity=0.01)
    .encode(
        x="x:Q",
        y="25th:Q",
        y2="75th:Q",
    )
)
chart_3_beta_median = (
    alt.Chart(df_2_Beta)
    .mark_line(color="red", opacity=0.5, strokeDash=[1, 1])
    .encode(
        y="median",
        x="x:Q",
    )
)
chart_3_beta_scatter_Df = pd.DataFrame(
    {
        "Average copy number (Log)": TE_2_ACN_2,
        "Epistasis Coefficient": epistasisCoefficient_2,
    }
)
chart_3_beta_scatter = (
    alt.Chart(chart_3_beta_scatter_Df)
    .mark_point(color=alpha_2_cols[0], opacity=0.01)
    .encode(
        x="Epistasis Coefficient",
        y="Average copy number (Log)",
    )
)
chart_3 = (
    chart_3_beta_0
    + chart_3_beta_1
    + chart_3_beta_2
    + chart_3_beta_scatter
    + chart_3_beta_median
)

# Custom label
text = (
    alt.Chart({"values": [{}]})
    .mark_text(align="left", baseline="top")
    .encode(
        x=alt.value(5),  # pixels from left
        y=alt.value(5),  # pixels from top
        text=alt.value(["Line 2"]),
    )
)

box = (
    alt.Chart({"values": [{}]})
    .mark_rect(stroke="black", color="orange")
    .encode(x=alt.value(3), x2=alt.value(50), y=alt.value(3), y2=alt.value(30))
)


chart = chart_1 + chart_2 + chart_3 + text + box
chart.properties(
    height=400,
    width=400,
    title="TE 1 under epistasis coefficients at generation 4000",
)

NameError: name 'epistasisCoefficient_1' is not defined